# Consolidating CDC data

Data was collected from fluvaxview data on the general population: https://www.cdc.gov/flu/fluvaxview/interactive-general-population.htm

Report data were downloaded for years 2013-14, 2014-15, 2015-16, 2016-17, and 2017-18





In [2]:
import pandas as pd
import numpy as np

In [3]:
def process_raw_fluvaxview(data_frame):
    """
    This will process the raw data that was generated from fluvaxview
    """
    
    
    data_frame.iloc[0,:] = data_frame.iloc[0,:].fillna(method='ffill')
    
    sample_data = data_frame

    to_fill = []
    for val in sample_data.columns:
        if 'Unnamed:' not in val:
            to_fill.append(val)
        else:
            to_fill.append(np.nan)

    sample_data.columns = pd.Series(data=to_fill).fillna(method='ffill')


    index = sample_data.index
    sample_data['index_place'] = index
    index = sample_data['index_place']
    index[0] = 'race_or_age' 
    sample_data['index_placeholder'] = pd.Series(index)
    sample_data = sample_data.set_index('index_placeholder')

#     return sample_data
    sample_data = sample_data.drop(labels=['index_place'], axis=1)


    sample_data = sample_data.transpose()


    months = ['Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec', 'Jan', 'Feb', 'Mar', 'Apr', 'May']

    month_column = []
    thing_to_fill_in = None
    for name in sample_data.Names:
        if name in months:
            thing_to_fill_in = name
        month_column.append(thing_to_fill_in)

    exp_column = []
    for name in sample_data.Names:
        if name in months:
            exp_column.append('avg')
        else:
            exp_column.append(name)


    sample_data['month'] = month_column
    sample_data['stats'] = exp_column


    sample_data = sample_data.drop(labels='Names', axis=1)


    regions = ['Alabama', 'Alaska', 'Arizona', 'Arkansas', 'California',
           'Colorado', 'Connecticut', 'Delaware', 'District of Columbia',
           'Florida', 'Georgia', 'Hawaii', 'Idaho', 'Illinois', 'Indiana', 'Iowa',
           'Kansas', 'Kentucky', 'Louisiana', 'Maine', 'Maryland', 'Massachusetts',
           'Michigan', 'Minnesota', 'Mississippi', 'Missouri', 'Montana',
           'Nebraska', 'Nevada', 'New Hampshire', 'New Jersey', 'New Mexico',
           'New York', 'North Carolina', 'North Dakota', 'Ohio', 'Oklahoma',
           'Oregon', 'Pennsylvania', 'Rhode Island', 'South Carolina',
           'South Dakota', 'Tennessee', 'Texas', 'Utah', 'Vermont', 'Virginia',
           'Washington', 'West Virginia', 'Wisconsin', 'Wyoming', 'United States',
           'Region  1', 'Region  2', 'Region  3', 'Region  4', 'Region  5',
           'Region  6', 'Region  7', 'Region  8', 'Region  9', 'Region 10',
           'HP 2020 Target']
    
#     return sample_data

    sample_data = sample_data.melt(value_vars=regions, id_vars=['month', 'stats', 'race_or_age'])
    
    return sample_data

In [4]:
data_2013 = pd.read_excel('../fluvaxdata/2013-2014 flu data.xlsx')
data_2014 = pd.read_excel('../fluvaxdata/2014-2015 flu data.xlsx')
data_2015 = pd.read_excel('../fluvaxdata/2015-2016 flu data.xlsx')
data_2016 = pd.read_excel('../fluvaxdata/2016-2017 flu data.xlsx')
data_2017 = pd.read_excel('../fluvaxdata/2017-2018 flu data.xlsx')

data_2013 = process_raw_fluvaxview(data_2013)
data_2014 = process_raw_fluvaxview(data_2014) 
data_2015 = process_raw_fluvaxview(data_2015) 
data_2016 = process_raw_fluvaxview(data_2016) 
data_2017 = process_raw_fluvaxview(data_2017) 

data_2013['year'] = '2013-2014'
data_2014['year'] = '2014-2015'
data_2015['year'] = '2015-2016'
data_2016['year'] = '2016-2017'
data_2017['year'] = '2017-2018'

data = data_2015.append(data_2016)
data = data.append(data_2017)
data = data.append(data_2014)
data = data.append(data_2013)

data.columns = ['month', 'stats', 'race_or_age', 'state_or_region', 'value', 'year']

data.head()

,month,stats,race_or_age,state_or_region,value,year
0,Jul,avg,≥6 months,Alabama,0.9,2015-2016
1,Jul,LL,≥6 months,Alabama,0.5,2015-2016
2,Jul,UL,≥6 months,Alabama,1.3,2015-2016
3,Jul,CI,≥6 months,Alabama,(±0.4),2015-2016
4,Jul,SAMPLE,≥6 months,Alabama,7822,2015-2016


In [5]:
data.to_csv('../fluvaxdata/2013-2018_consolidated_flu_data.csv')

# Processing consolidated CDC data

Read the CDC consolidated dataframe, and set up some preliminary variables

In [6]:
import pandas as pd
cdc = pd.read_csv("../fluvaxdata/2013-2018_consolidated_flu_data.csv")

In [7]:
state_names = ["Alabama","Alaska","Arizona","Arkansas","California","Colorado",
  "Connecticut","Delaware","Florida","Georgia","Hawaii","Idaho","Illinois",
  "Indiana","Iowa","Kansas","Kentucky","Louisiana","Maine","Maryland",
  "Massachusetts","Michigan","Minnesota","Mississippi","Missouri","Montana",
  "Nebraska","Nevada","New Hampshire","New Jersey","New Mexico","New York",
  "North Carolina","North Dakota","Ohio","Oklahoma","Oregon","Pennsylvania",
  "Rhode Island","South Carolina","South Dakota","Tennessee","Texas","Utah",
  "Vermont","Virginia","Washington","West Virginia","Wisconsin","Wyoming"]

As we see from the sample, there are multiple statistics and regions being considered

In [8]:
cdc.sample(20)

,Unnamed: 0,month,stats,race_or_age,state_or_region,value,year
92800,30430,Oct,avg,≥65 years,New Jersey,41.0,2016-2017
102145,39775,Sep,avg,5-12 years,South Carolina,13.8,2016-2017
280771,31291,Aug,avg,50-64 years,New Mexico,0.8,2013-2014
284828,35348,Jul,avg,"White only, non-Hispanic",Ohio,NR †,2013-2014
197277,10167,Apr,UL,13-17 years,Georgia,48.8,2014-2015
238958,51848,Feb,CI,18-64 years,Region 1,(±1.8),2014-2015
154076,29336,Nov,LL,18-49 years not at high risk,New Hampshire,18.8,2017-2018
184005,59265,Jan,avg,"Black only, non-Hispanic",Region 8,38.8,2017-2018
56725,56725,Nov,avg,≥18 years,Region 6,32.2,2015-2016
301132,51652,Jul,LL,≥18 years,Region 1,0.2,2013-2014


For our purposes, we only need the "avg" statistic, which indicates the percentage of patients covered by that date in the season. We only need state-by-state data:

In [5]:
cdc_avg = cdc[cdc["state_or_region"].isin(state_names) * cdc["stats"]=="avg"]

In [6]:
cdc_avg.sample(20)

,Unnamed: 0,month,stats,race_or_age,state_or_region,value,year
254350,4870,Mar,avg,Hispanic,California,NR,2013-2014
276616,27136,Nov,avg,18-64 years not at high risk,Nebraska,33.1,2013-2014
208640,21530,Dec,avg,≥65 years,Massachusetts,59.6,2014-2015
9750,9750,Oct,avg,"Black only, non-Hispanic",Florida,18.3,2015-2016
78400,16030,Dec,avg,5-12 years,Kansas,49.0,2016-2017
251921,2441,Dec,avg,18-64 years not at high risk,Arizona,21.3,2013-2014
2540,2540,Sep,avg,18-49 years at high risk,Arizona,7.4,2015-2016
259735,10255,Apr,avg,18-64 years,Georgia,30.9,2013-2014
37435,37435,Feb,avg,"White only, non-Hispanic",Oregon,41.0,2015-2016
98690,36320,Nov,avg,50-64 years,Oklahoma,42.4,2016-2017


In [7]:
cdc_avg.shape

(49500, 7)

We need to remove all the "non-reported" values, and convert the rest to float.

In [8]:
#reomve "NR" values
cdc_avg_cleaned = cdc_avg[(cdc_avg["value"] != "NR †") & (cdc_avg["value"] != "NR *") & (cdc_avg["value"] != "NR")]
cdc_avg_cleaned["value_flt"] = cdc_avg_cleaned["value"].apply(float)
cdc_avg_cleaned.drop(columns=["Unnamed: 0"], inplace=True)

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/anaconda3/lib/python3.6/site-packages/pandas/core/frame.py:3694: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


In [9]:
cdc_avg_cleaned.sample(20)

,month,stats,race_or_age,state_or_region,value,year,value_flt
190880,Jan,avg,"White only, non-Hispanic",Arkansas,46.5,2014-2015,46.5
141695,Oct,avg,6 months - 4 years,Kentucky,35.0,2017-2018,35.0
283047,Jan,avg,"Other or multiple races, non-Hispanic",North Carolina,43.5,2013-2014,43.5
75710,Jan,avg,18-64 years not at high risk,Illinois,29.5,2016-2017,29.5
202885,Apr,avg,Hispanic,Iowa,56.6,2014-2015,56.6
138915,Mar,avg,≥18 years,Indiana,32.2,2017-2018,32.2
152315,Nov,avg,"Black only, non-Hispanic",Nebraska,36.3,2017-2018,36.3
173115,Jan,avg,"Black only, non-Hispanic",West Virginia,43.6,2017-2018,43.6
277219,Nov,avg,6 months - 17 years,Nevada,39.0,2013-2014,39.0
263352,Mar,avg,≥6 months,Indiana,40.9,2013-2014,40.9


We now want just the full >= 6-months dataset

In [10]:
cdc_allpopulation = cdc_avg_cleaned[cdc_avg_cleaned.race_or_age == "≥6 months"]

Now compare the number of rows to the number of all possible data points for 5 years. We expect less due to some non-reported values.

In [11]:
cdc_allpopulation.shape

(2665, 7)

In [12]:
5*12*50

3000

In [13]:
cdc_allpopulation.sample(20)

,month,stats,race_or_age,state_or_region,value,year,value_flt
156420,Jul,avg,≥6 months,New York,0.7,2017-2018,0.7
10915,Dec,avg,≥6 months,Hawaii,40.7,2015-2016,40.7
45575,Feb,avg,≥6 months,Virginia,47.6,2015-2016,47.6
81210,Jan,avg,≥6 months,Maine,46.3,2016-2017,46.3
93085,Dec,avg,≥6 months,New Mexico,41.6,2016-2017,41.6
151490,Nov,avg,≥6 months,Nebraska,38.8,2017-2018,38.8
218790,Jul,avg,≥6 months,New York,0.7,2014-2015,0.7
109925,Feb,avg,≥6 months,West Virginia,48.2,2016-2017,48.2
66370,Mar,avg,≥6 months,California,47.7,2016-2017,47.7
67365,Apr,avg,≥6 months,Colorado,49.7,2016-2017,49.7


We can now convert the dates to a more convenient time-stamp object, taking into account they way a 'season' is defined by CDC:

In [14]:
def convertToTimestamp(monthCol, yearCol):
    dateCol = []
    for (month, year) in zip(monthCol, yearCol):
        if (month in ["Jul", "Aug", "Sep", "Oct", "Nov", "Dec"]):
            yr = year[0:4]
        else:
            yr = year[-4:]
        dateCol.append(pd.Timestamp.strptime(month + " 15 " + yr, "%b %d %Y"))
        
    return dateCol

In [15]:
cdc_allpopulation["time"] = convertToTimestamp(cdc_allpopulation["month"].values, cdc_allpopulation["year"].values)

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


Do some renaming of columns, and save the final dataframe

In [16]:
cdc_allpopulation.rename(columns={'value_flt': 'mean_pct', 'state_or_region': 'state'}, inplace=True)

/anaconda3/lib/python3.6/site-packages/pandas/core/frame.py:3778: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  return super(DataFrame, self).rename(**kwargs)


In [17]:
cdc_allpopulation.sample(20)

,month,stats,race_or_age,state,value,year,mean_pct,time
207905,Aug,avg,≥6 months,Massachusetts,2.4,2014-2015,2.4,2014-08-15
193055,Aug,avg,≥6 months,Connecticut,1.6,2014-2015,1.6,2014-08-15
230675,Aug,avg,≥6 months,Utah,1.4,2014-2015,1.4,2014-08-15
173290,Mar,avg,≥6 months,Wisconsin,39.3,2017-2018,39.3,2018-03-15
196030,Sep,avg,≥6 months,Florida,9.1,2014-2015,9.1,2014-09-15
149505,Oct,avg,≥6 months,Missouri,26.6,2017-2018,26.6,2017-10-15
90100,Sep,avg,≥6 months,Nevada,9.1,2016-2017,9.1,2016-09-15
221770,Sep,avg,≥6 months,Ohio,9.2,2014-2015,9.2,2014-09-15
94095,Apr,avg,≥6 months,New York,49.2,2016-2017,49.2,2017-04-15
47565,Apr,avg,≥6 months,West Virginia,49.2,2015-2016,49.2,2016-04-15


In [18]:
cdc_allpopulation.to_csv("cdc_average_bystate_2013-2017.csv")